In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%run figures.py

In [ ]:
%matplotlib notebook

import matplotlib as mpl
from matplotlib import pyplot as plt
from cmcrameri import cm
import numpy as np
import copy
import string
mpl.rcParams.update(mpl.rcParamsDefault)

mpl.rcParams['axes.linewidth'] = 3
mpl.style.use('seaborn-poster')

time_steps = 25
i = 0
file_num=0
fontsize = 24
time = 19


fig, axes = plt.subplots(1,2, figsize=(19,9))
axis = axes[0]
axis2 = axes[1]
#axes = axes.flatten()

category = 'volume'
plot_vars = ['Calcite'] * 4

#Cell vol /cm^3
cell_vol = 200 * 100**2
#Calcite molar volume cm^3/mol
calcite_mv = 36.934
#Calcite molar mass g/mol
# Calcite density
vf_to_mols = cell_vol / calcite_mv

scale_factors = (vf_to_mols, vf_to_mols, vf_to_mols, vf_to_mols)

axis2.grid(False)
axis2.tick_params(length=8, width=4)
axis2.set_ylabel('Calcite / moles', fontweight='bold')
axis2.set_xlabel('Concentration / mM', fontweight='bold')
#axis2.set_ylim(8000, 13000)
axis2.text(-0.05, 1.05, f'({string.ascii_uppercase[1]})', transform=axis2.transAxes, size=20, weight='bold')
#axis.set_xlim(0,175)
for tick in axis2.xaxis.get_major_ticks():
    tick.label1.set_fontweight('bold')
    tick.label2.set_fontweight('bold')
for tick in axis2.yaxis.get_major_ticks():
    tick.label1.set_fontweight('bold')
    tick.label2.set_fontweight('bold')
#axis.invert_yaxis()

axins = axis2.inset_axes([0.6, 0.65, 0.3, 0.3])

for dataset, ls, temperature in zip(datasets, linestyles, temperatures):

    iterables = zip(np.arange(len(plot_vars)), plot_vars, legend_titles, scale_factors, range_sets, labels, leg_entries)
    # Axis level loop
    for j, plot_var, legend_title, scale, range_set, label_a, leg_entry in iterables:
        # Line set loop (i.e. groups of lines sharing a property)
        # Line instance loop
        index = np.arange(range_set[0], range_set[1]+1)
        y = []
        x = copy.deepcopy(label_a)
        pop_list = []

        for n, ind in enumerate(index):
            try:
                val = dataset[ind].results[category].isel(time=time, Y=0, Z=0)[plot_var].sum(dim='X').to_numpy() * scale
                pre_burn = dataset[ind].results[category].isel(time=11, Y=0, Z=0)[plot_var].sum(dim='X').to_numpy() * scale
                pre_burn = 0
                y.append(val - pre_burn)
            except:
                pop_list.append(n)
                continue

        pop_list.sort(reverse=True)
        for q in pop_list:
            x.pop(q)

        if j == 3:
            axins.plot(x,y, label=f'{leg_entry}', linestyle=ls, c='r')
            axis2.plot(20,20000, label=f'{leg_entry}')
            secaxins = axins.secondary_yaxis('right', functions=(c_moles_to_flux, c_flux_to_moles))
            axins.tick_params(length=8, width=4)
            secaxins.tick_params(length=8, width=4)
            for tick in axins.yaxis.get_major_ticks():
                tick.label1.set_fontweight('bold')
                tick.label2.set_fontweight('bold')
            for tick in secaxins.yaxis.get_major_ticks():
                tick.label1.set_fontweight('bold')
                tick.label2.set_fontweight('bold')
            for tick in axins.xaxis.get_major_ticks():
                tick.label1.set_fontweight('bold')
                tick.label2.set_fontweight('bold')
        else:
            axis2.plot(x, y, label= f'{leg_entry}', linestyle=ls)
            secax = axis2.secondary_yaxis('right', functions=(c_moles_to_flux, c_flux_to_moles))
            secax.set_ylabel('Global C flux / Tg yr$\mathbf{^{-1}}$', fontweight='bold')
            secax.tick_params(length=8, width=4)
            for tick in secax.yaxis.get_major_ticks():
                tick.label1.set_fontweight('bold')
                tick.label2.set_fontweight('bold')
        axis2.legend(title='Seawater component', prop={'weight': 'bold', 'size': 15}, title_fontproperties={'weight': 'bold', 'size': 15}, framealpha=0, loc=2)
        axis2.set_ylim(15000, 24500)
        axins.set_ylim(15500, 24500)

fig.tight_layout()

fig.savefig('output/calc_v_cations.png', dpi=300)
fig.show()
